# Import Modules

In [1]:
import os
import pandas as pd
from credentials import *
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine, text

c:\Users\ASANTE\miniconda3\envs\SNOWFLAKE\lib\site-packages\snowflake\connector\options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


# Helper Functions

In [2]:
def make_conn(**db_parameters):
    """
    Composes a SQLAlchemy connect string from the given database connection parameters.

    https://github.com/snowflakedb/snowflake-sqlalchemy#escaping-special-characters-such-as---signs-in-passwords
    
    """
    try:
        engine = create_engine(URL(**db_parameters))
        return engine.connect()
    
    except snowflake.connector.errors.ProgrammingError as e:
        return f"Error: {e}"
    
    finally:
        engine.connect().close()
        engine.dispose()

# Establsih Connection

In [3]:
conn = make_conn(role=ROLE, account=ACCOUNT_ID, user=USERNAME, password=PASSWORD, database='', schema='', warehouse=WAREHOUSE)

# Create Database and Schema

Create Database

In [4]:
db_name = "aws_metro_bike_share"

create_database_query = f"""
                         CREATE DATABASE IF NOT EXISTS {db_name}
                         """

conn.execute(text(create_database_query))

Create Schema

In [5]:
conn = make_conn(role=ROLE, account=ACCOUNT_ID, user=USERNAME, password=PASSWORD, database=db_name, schema='', warehouse=WAREHOUSE)

schema_name = "aws_metro_bike_share_schema"

create_schema_query = f"""
                       CREATE SCHEMA IF NOT EXISTS {schema_name}
                       """

conn.execute(text(create_schema_query))

# Create AWS Integration

In [6]:
conn = make_conn(role=ROLE, account=ACCOUNT_ID, user=USERNAME, password=PASSWORD, database=db_name, schema=schema_name, warehouse=WAREHOUSE)
integration_name = 'aws_s3_integration'
integration_comment = 'AWS Storage Integration'

aws_integration_query = f"""
                         CREATE OR REPLACE STORAGE INTEGRATION {integration_name}
                         TYPE = EXTERNAL_STAGE
                         STORAGE_PROVIDER = 'S3'
                         ENABLED = TRUE
                         STORAGE_AWS_ROLE_ARN = '{AWS_ARN}'
                         STORAGE_ALLOWED_LOCATIONS = ('{AWS_LOCATION}')
                         COMMENT = '{integration_comment}'

                        """

conn.execute(text(aws_integration_query))
conn.execute(text("""SHOW INTEGRATIONS like '%s3%'""")).fetchall()

[('AWS_S3_INTEGRATION', 'EXTERNAL_STAGE', 'STORAGE', 'true', 'AWS Storage Integration', datetime.datetime(2023, 7, 24, 13, 41, 9, 586000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>))]